In [1]:
import pandas as pd
import json
import engin
import models
import random

In [2]:
with open('/Users/daniil/Documents/Работа/Платформа анализа данных/Прогноз/Auto_forecast/test_data_10000.json', "r", encoding="utf8") as f:
    data = json.load(f)

In [3]:
passed, failed = engin.Filter_for_dataset(data)
print('passed -', len(passed))
print('failed -', len(failed))

passed - 7730
failed - 2270


In [4]:
df = pd.DataFrame(passed)

Разделим данные по частотностям:

In [5]:
# Инициализируем три пустых списка
ye_list = []
q_list = []
me_list = []

# Проходим по исходному списку
for item in passed:
    freq = item.get("pandas_frequency")
    if freq == "YE":
        ye_list.append(item)
    elif freq == "Q":
        q_list.append(item)
    elif freq == "ME":
        me_list.append(item)

print('Годовые данные: ',len(ye_list))
print('Квартальные данные: ',len(q_list))
print('Месячные данные: ',len(me_list))

Годовые данные:  3178
Квартальные данные:  106
Месячные данные:  4446


Так как функция автопрогноза требудет больших вычислительных мощностей, ее тестирование будет для случайно выбранных рядов из каждой категории

In [6]:
# Фиксируем seed для воспроизводимости
random.seed(42+2)

sample_ye = random.sample(ye_list, 30)
sample_q  = random.sample(q_list, 30)
sample_me = random.sample(me_list, 30)

In [7]:
import warnings, traceback

In [8]:
def testing(data: list,
            Deep_research: bool):
    
    successful_results, raise_dic, warns = {}, {}, []
    
    for i in range(len(data)):
        df = pd.DataFrame(data)
        slice_id = df.loc[i, 'slice_id']
        Data = pd.DataFrame(df.loc[i, 'observations'])

        with warnings.catch_warnings(record=True) as wlist:
                warnings.simplefilter("always")                  # перехватываем все предупреждения
                try:
                    out = engin.Auto_forecast(data[i], Deep_research)
                    successful_results[slice_id] = out
                except Exception as e:
                    # Сохраняем возникшие исключения
                    raise_dic[slice_id] = {
                    "Error_test": e, 
                    "type": type(e).__name__,
                    "msg": str(e),
                    "traceback": traceback.format_exc()
                }
            # Сохраняем предупреждения из этого прогона
                for w in wlist:
                    warns.append({
                        "slice_id": slice_id,
                        "category": w.category.__name__,
                        "message": str(w.message),
                        "filename": w.filename,
                        "lineno": w.lineno,
                    })

        result = {
            'successful_results' : successful_results,
            'raise_dic' : raise_dic,
            'warns' : warns
        }
    return result


In [9]:
res_test_YE_F = testing(sample_ye, False)
res_test_YE_T = testing(sample_ye, True)
print('Статистика: ')
print('--------------------------------------------------------')
print('Успешно c F: ', len(res_test_YE_F['successful_results']))
print('Исключения c F: ', len(res_test_YE_F['raise_dic']))
print('Предупреждения c F: ', len(res_test_YE_F['warns']))
print('--------------------------------------------------------')
print('Успешно c T: ', len(res_test_YE_T['successful_results']))
print('Исключения c T: ', len(res_test_YE_T['raise_dic']))
print('Предупреждения c T: ', len(res_test_YE_T['warns']))

Статистика: 
--------------------------------------------------------
Успешно c F:  15
Исключения c F:  15
Предупреждения c F:  3280
--------------------------------------------------------
Успешно c T:  15
Исключения c T:  15
Предупреждения c T:  9263


In [10]:
#res_test_YE_F['raise_dic'] # все годовы данные не прошли по причине 'Слишком короткий временной ряд!'

In [16]:
pd.DataFrame(res_test_YE_F['raise_dic'])#.iloc[3,0]

,ef5c052f-33ee-44eb-b3b4-37c6517e22ae,d05acaa3-dc23-469f-9b22-2741875e8d99,466b1225-efcc-4fa5-a8d0-507ef028cfd4,978dd99d-5af3-4c3f-9943-bb8cbb18aa7d,d254dbc9-992a-4534-a2e6-b8f9c9f7f306,2dd83d41-82f2-46c7-ae4e-be015957f08e,43724219-6745-4aeb-8eae-7da83439ecea,ea869b99-d143-4e9f-9526-48a2645e3b32,159dbf01-1c24-4e0a-a8f9-90c585b2b87c,00d22a39-5ab5-4669-a877-644352c69113,c05d0635-11e8-42fe-8a73-80890a640de1,3f7583b0-a667-430a-b656-e76093149054,8c3d8d2e-9284-44e6-8528-bfc5dcd68a49,3c2443e9-fdf7-4890-b284-2aadc962113a,49037ead-3e08-4adc-8b4e-f435621a1e06
Error_test,Слишком короткий временной ряд!,Слишком короткий временной ряд!,Слишком короткий временной ряд!,Слишком короткий временной ряд!,Слишком короткий временной ряд!,Слишком короткий временной ряд!,Слишком короткий временной ряд!,Слишком короткий временной ряд!,Слишком короткий временной ряд!,Слишком короткий временной ряд!,Слишком короткий временной ряд!,Слишком короткий временной ряд!,Слишком короткий временной ряд!,Слишком короткий временной ряд!,Слишком короткий временной ряд!
type,AssertionError,AssertionError,AssertionError,AssertionError,AssertionError,AssertionError,AssertionError,AssertionError,AssertionError,AssertionError,AssertionError,AssertionError,AssertionError,AssertionError,AssertionError
msg,Слишком короткий временной ряд!,Слишком короткий временной ряд!,Слишком короткий временной ряд!,Слишком короткий временной ряд!,Слишком короткий временной ряд!,Слишком короткий временной ряд!,Слишком короткий временной ряд!,Слишком короткий временной ряд!,Слишком короткий временной ряд!,Слишком короткий временной ряд!,Слишком короткий временной ряд!,Слишком короткий временной ряд!,Слишком короткий временной ряд!,Слишком короткий временной ряд!,Слишком короткий временной ряд!
traceback,"Traceback (most recent call last):\n File ""/v...","Traceback (most recent call last):\n File ""/v...","Traceback (most recent call last):\n File ""/v...","Traceback (most recent call last):\n File ""/v...","Traceback (most recent call last):\n File ""/v...","Traceback (most recent call last):\n File ""/v...","Traceback (most recent call last):\n File ""/v...","Traceback (most recent call last):\n File ""/v...","Traceback (most recent call last):\n File ""/v...","Traceback (most recent call last):\n File ""/v...","Traceback (most recent call last):\n File ""/v...","Traceback (most recent call last):\n File ""/v...","Traceback (most recent call last):\n File ""/v...","Traceback (most recent call last):\n File ""/v...","Traceback (most recent call last):\n File ""/v..."


In [12]:
res_test_Q_F = testing(sample_q, False)
res_test_Q_T = testing(sample_q, True)
print('Статистика: ')
print('--------------------------------------------------------')
print('Успешно c F: ', len(res_test_Q_F['successful_results']))
print('Исключения c F: ', len(res_test_Q_F['raise_dic']))
print('Предупреждения c F: ', len(res_test_Q_F['warns']))
print('--------------------------------------------------------')
print('Успешно c T: ', len(res_test_Q_T['successful_results']))
print('Исключения c T: ', len(res_test_Q_T['raise_dic']))
print('Предупреждения c T: ', len(res_test_Q_T['warns']))

Статистика: 
--------------------------------------------------------
Успешно c F:  26
Исключения c F:  4
Предупреждения c F:  6908
--------------------------------------------------------
Успешно c T:  27
Исключения c T:  3
Предупреждения c T:  19911


In [13]:
pd.DataFrame(res_test_Q_T['raise_dic'])

,ac3b78bc-8df8-4478-8938-4920e61ca2e8,e13913ca-b232-44f7-9cd2-6352356e5b05,2e73a7d3-6d9e-4699-8d91-6dfcb9355314
Error_test,Слишком короткий временной ряд!,Слишком короткий временной ряд!,Слишком короткий временной ряд!
type,AssertionError,AssertionError,AssertionError
msg,Слишком короткий временной ряд!,Слишком короткий временной ряд!,Слишком короткий временной ряд!
traceback,"Traceback (most recent call last):\n File ""/v...","Traceback (most recent call last):\n File ""/v...","Traceback (most recent call last):\n File ""/v..."


In [14]:
res_test_ME_F = testing(sample_me, False)
res_test_ME_T = testing(sample_me, True)
print('Статистика: ')
print('--------------------------------------------------------')
print('Успешно c F: ', len(res_test_ME_F['successful_results']))
print('Исключения c F: ', len(res_test_ME_F['raise_dic']))
print('Предупреждения c F: ', len(res_test_ME_F['warns']))
print('--------------------------------------------------------')
print('Успешно c T: ', len(res_test_ME_T['successful_results']))
print('Исключения c T: ', len(res_test_ME_T['raise_dic']))
print('Предупреждения c T: ', len(res_test_ME_T['warns']))

Статистика: 
--------------------------------------------------------
Успешно c F:  25
Исключения c F:  5
Предупреждения c F:  8679
--------------------------------------------------------
Успешно c T:  25
Исключения c T:  5
Предупреждения c T:  43977


In [15]:
pd.DataFrame(res_test_ME_T['raise_dic'])

,59a67716-1ee4-4c3a-8630-a9825d5e80c5,971d5c1e-01b5-4f16-a86f-7883f2bd6529,312bd3a8-33e2-4573-81c4-fa8f7b1e864d,540c10b6-0a86-4ce3-8dbe-ec94a1f7bce9,ea234156-6e9e-45c4-9d89-2ac8b80624d1
Error_test,Слишком короткий временной ряд!,Слишком короткий временной ряд!,Слишком короткий временной ряд!,Слишком короткий временной ряд!,Слишком короткий временной ряд!
type,AssertionError,AssertionError,AssertionError,AssertionError,AssertionError
msg,Слишком короткий временной ряд!,Слишком короткий временной ряд!,Слишком короткий временной ряд!,Слишком короткий временной ряд!,Слишком короткий временной ряд!
traceback,"Traceback (most recent call last):\n File ""/v...","Traceback (most recent call last):\n File ""/v...","Traceback (most recent call last):\n File ""/v...","Traceback (most recent call last):\n File ""/v...","Traceback (most recent call last):\n File ""/v..."
